In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import copy
import pandas as pd

In [ ]:
class Film: 
  def __init__():
      Title = None
      IMDB_Rating = None
      Reelgood_Rating = None
      Genres = []
      Year = None
      Tags = []
      Country = None
      Source = None
      Rated = None
      Duration = None
      Description = None
      Director = None
      Employees = []

filmObj = {
      "Title": None,
      "IMDB_Rating": None,
      "Reelgood_Rating": None,
      "Genres": [],
      "Year": None,
      "Tags": [],
      "Country": None,
      "Source": None,
      "Rated": None,
      "Duration": None,
      "Description": None,
      "Director": None,
      "Employees": [],
}

In [ ]:
with open('/kaggle/input/link-films-crawl/link_movies_netflix.txt') as f:
    links = [line.rstrip() for line in f]
links

In [ ]:
arr_film = []
for i in range(3013, len(links)):
  link = links[i]
  film = copy.deepcopy(filmObj)

  try:
    with requests.get(link) as page:
      soup = BeautifulSoup(page.content, 'html.parser')
      film['Title'] = soup.find(class_="css-of585f e14injhv7")
      if film['Title'] != None:
        film['Title'] = film['Title'].get_text()

      div_info = soup.find(class_="css-1su90nd ey4ir3j0")

      # GET SCORE
      film['IMDB_Rating'] = div_info.find(class_="css-xmin1q ey4ir3j3").get_text()
      film['Reelgood_Rating'] = div_info.find(class_="css-xmin1q ey4ir3j8")
      if film['Reelgood_Rating'] != None:
        film['Reelgood_Rating'] = film['Reelgood_Rating'].get_text()

      # GET INFO BY TAG A
      list_info = div_info.find_all(class_="css-10wrqt0")
      for info in list_info:
        type_info = info.get("href")
        if('/genre' in type_info):
          film['Genres'].append(info.get_text())
        elif '/year' in type_info:
          film['Year'] = info.get_text()
        elif '/list' in type_info: 
          film['Tags'].append(info.get_text())
        elif '/origin' in type_info:
          film['Country'] = info.get_text()
        elif '/source' in type_info:
          film['Source'] = info.get_text()

      # GET INFO BY TAG META
      film['Rated'] = div_info.find('meta', itemprop='contentRating')
      if film['Rated'] != None:
        film['Rated'] = film['Rated'].get("content", None)
      film['Duration'] = div_info.find('meta', itemprop='duration')
      if film['Duration'] != None: 
        film['Duration'] = film['Duration'].get("content", None)

      # GET DESCRIPTION
      film['Description'] = soup.find('p', itemprop="description").get_text()

      # GET CAST & CREW
      list_employee = soup.find_all('h3', itemprop="name", class_="css-11yr18h egg5eqo2")
      # GET DIRECTOR
      if len(list_employee) > 0:
        film['Director'] = list_employee[0].get_text()
        #GET EMPLOYEE
        for i in range(1, len(list_employee)):
          employee = list_employee[i]
          film['Employees'].append(employee.get_text())

      print(link, " done")
      arr_film.append(film)

  except (NameError, AttributeError) as e:
    print(e)
    continue;
f.close()


In [ ]:
df = pd.DataFrame(arr_film)
df

In [ ]:
%cd /kaggle/working

In [ ]:
df.to_csv('mycsvfile7.csv',index=False, encoding="utf-8")